**EXECUTE THIS NOTEBOOK IN COOLAB**

In [ ]:
# importar librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Procesamiento de Datos

In [ ]:
#!curl https://raw.githubusercontent.com/LuisFelipePoma/PC2-IA/main/src/model/heart.csv -o heart.csv

In [ ]:
# obtener el conjunto de datos
df = pd.read_csv('heart.csv')

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.head(5)

In [ ]:
# see if the data is balance in a plot
df["output"].value_counts().plot(kind='bar')

In [ ]:
# Verify if there are any null values
print(df.isna().sum())

In [ ]:
df.describe()

## Transformación de Datos


In [ ]:
# ---------------------> DROP DATA <---------------------

df.drop(
    columns=["chol", "fbs", "trtbps", "restecg", "slp", "oldpeak", "caa"],
    inplace=True,
)

# df = df.loc[df["thall"] != 0]

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
# graph of the corr
import seaborn as sns
plt.figure(figsize=(10, 10))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")
plt.show()

# Modelate

In [ ]:
df.head()

## Split Data

In [ ]:
# Split the dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    ConfusionMatrixDisplay,
)


X_train, X_test, y_train, y_test = train_test_split(
    df.drop("output", axis=1),
    df["output"],
    random_state=42,
    test_size=0.2,
)
# Path to save the models
PATH = "../client/public/models/"

In [ ]:
# Eliminamos el Id del pasajero en el conjunto de datos de prueba
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Transfom Models

In [ ]:
%pip install skl2onnx
%pip install tensorflowjs

In [ ]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
# import tensorflowjs as tfjs

def save_model(model, file):
    initial_type = [("input", FloatTensorType([None, X_train.shape[1]]))]
    onx = convert_sklearn(model, initial_types=initial_type, options={type(model): {'zipmap': False}})
    with open(PATH + file + ".onnx", "wb") as f:
        f.write(onx.SerializeToString())

## Model Perceptron


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input

In [ ]:
perceptron = Sequential()

perceptron.add(Input(shape=(X_train.shape[1],)))
perceptron.add(Dense(units=1, activation="sigmoid"))
perceptron.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
perceptron.summary()


In [ ]:
history = perceptron.fit(X_train, y_train, epochs=250, batch_size=8, verbose=1, validation_data=(X_test, y_test))
# save model and architecture to single file
#perceptron.save(PATH + "perceptron.h5")
tfjs.converters.save_keras_model(perceptron, "perceptron_model")

## Model Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
save_model(gnb, "model_gnb")
y_pred = gnb.predict(X_test)
# print accuracy
acc_gnb = accuracy_score(y_test, y_pred)
print(acc_gnb)

### CNN

In [ ]:
# Import lib for convolutional neural network
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout

In [ ]:
modelconv = Sequential()
# modelconv.add(tf.keras.Input(shape=(10,)))
modelconv.add(Conv1D(4, 3, activation="relu", input_shape=(X_train.shape[1], 1)))
modelconv.add(MaxPooling1D())
modelconv.add(Flatten())
modelconv.add(Dense(4, activation="relu"))
modelconv.add(Dropout(0.1))
modelconv.add(Dense(1, activation="sigmoid"))
modelconv.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
modelconv.summary()

In [ ]:
# Train the model
historyconv = modelconv.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=500, batch_size=8
)
# save model to .h5
#modelconv.save( PATH + "modelconv.h5")
tfjs.converters.save_keras_model(perceptron, "cnn_model")


In [ ]:
# graph of loss
plt.plot(historyconv.history['loss'])
plt.plot(historyconv.history['val_loss'])
plt.title('Loss vs Epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()